# <center>Punchlines as Mirrors: Social Attitudes, Politics, and Biases in *The New Yorker* Caption Contest</center>
<center><b>Humor reflecting society’s views, stereotypes, and political climate</b></center>
<br/><br/>
<center>
    <img src="data/newyorker_caption_contest_virgin/images/545.jpg" alt="New Yorker Cartoon" style="width:300px; height:auto; border-radius:5px; box-shadow: 0 4px 8px rgba(0,0,0,0.1);">
</center>

---

# <center>Milestone 2 : Table of content</center>

Imports, initialisations\
Path system
1. **Data preprocessing**
   - 1.1. Cleaning and preprocessing of the dataset
          Construction of a New Funny Metric
   - 1.2. Gathering of other datasets
   - 1.3. Executing the DataPreparation.ipynb file
   - 1.4. Descriptive statistic tasks

2. **Building usefull metrics**
    - 1.1. Similarity metric
    - 1.2. Funny-like metric

3. **Narrative Flow**
   - 2.1. **Axis 1:** Professions & politics → humor about authority and power, *“What are people laughing about?”*
   - 2.2. **Axis 2:** Humor in time → historical & contextual dimensions, *“When and why do jokes resonate?”*
   - 2.3. **Axis 3:** Social norms → gender roles & taboos, testing the limits of humor, *“What’s acceptable or not?”*
   - 2.4. **Axis 4:** Biases → explain psychological and cultural mechanisms behind why we laugh, *“Why do we find it funny?”*

4. **Conclusions of the milestone 2**


## Path system


To be consitent in our **path system**, a **centralized file** has been created in 'src\utils\paths.py' and contains all relative paths of the project.

In [1]:
from src.utils.paths import STORED_DATAPREP_PKL_PATH, STORED_PLOTSGUI_PKL_PATH, DATA_PREPARATION_PY_PATH


## Imports, initialisations

In [2]:
import pandas as pd
%matplotlib inline

In case an import cannot be imported run this:

In [3]:
import importlib
import src.utils.paths as p
importlib.reload(p)

<module 'src.utils.paths' from 'c:\\Users\\cyrie\\OneDrive\\Documents\\EPFL\\2025_MA_1\\ADA\\ada-2025-project-adacore42\\src\\utils\\paths.py'>

In [4]:
# -----------------------------------------------------------------------------
# Robust project root detection (works from notebook or script)
# -----------------------------------------------------------------------------
from pathlib import Path
import sys, os

# Detect root starting from this file or from notebook’s cwd
try:
    start_path = Path(__file__).resolve()
except NameError:
    start_path = Path.cwd()

root = start_path
while root != root.parent:
    # Check for any known project markers
    if any((root / marker).exists() for marker in [".git", "README.md", "results.ipynb", "README.txt"]):
        break
    root = root.parent

# Sanity check — fallback if nothing found
if not any((root / marker).exists() for marker in [".git", "README.md", "results.ipynb", "README.txt"]):
    print("⚠️ Project root not found — defaulting to current working directory")
    root = Path.cwd()

print(f"✅ Root folder detected at: {root}")

# Add project root to sys.path if not already
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

✅ Root folder detected at: c:\Users\cyrie\OneDrive\Documents\EPFL\2025_MA_1\ADA\ada-2025-project-adacore42


In [3]:
import os
from src.utils.paths import DATA_PREPARATION_PY_PATH
print(DATA_PREPARATION_PY_PATH.resolve())

/Users/todorovkatia/Documents/EPFL/MA3/ADA/Projet/ada-2025-project-adacore42/src/data/DataPreparation.py


In [4]:
from pathlib import Path
import sys
import os
import pickle

# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
try:
    root = Path(__file__).resolve().parent
except NameError:
    root = Path.cwd()  # fallback for Jupyter notebooks

while root.parent != root:
    if any((root / marker).exists() for marker in [".git", "README.md", "results.ipynb", "README.txt"]):
        break
    root = root.parent

# Fallback in case nothing found
if not any((root / marker).exists() for marker in [".git", "README.md", "results.ipynb", "README.txt"]):
    print("⚠️ Could not locate project root — defaulting to current working directory")
    root = Path.cwd()

print(f"✅ Root folder detected at: {root}")

# Ensure importability of the project
if str(root) not in sys.path:
    sys.path.insert(0, str(root))



✅ Root folder detected at: /Users/todorovkatia/Documents/EPFL/MA3/ADA/Projet/ada-2025-project-adacore42


# 1 Data preprocessing

## 1.1. Cleaning and preprocessing of the dataset

##### Concerning the CSV files for each contests, the steps were :
- **Concatenate all 385 CSV in a list of CSV**\
After having done initial analysis and data verification, we observed 2 contests were there were unconsistency. The contest 525 does not have any image nor CSV associated, and the contest 540 does have an image, but any CSV associated. We removed those contests from the dataset. 
- **Remove redundant columns (index and rank)**\
We could import directly with rank index:
        ```python pd.read_csv(f"../newyorker_caption_contest_virgin/data/{i}.csv", index_col=['rank'])```
        but since not all files have column rank it makes sense to import as it is and later remove the redundant column. 


- **Consistency verification**\
Test if there are any NaN, Since data not contain some values, we are searching the NaN and replacing. 
        ```python data.isnull().values.any():```
        return true if there is any value that is null from data 
        ```python dataA3[i].fillna('CAPTION_NOT_FOUND', inplace = True):```
        For dataframe i fill ALL na values with 'text'

##### Concerning the JSON file, the steps were :
- **Remove non-used columns**\
Only keeping 'metadata' columns of the json, since it contains the relevant datas
- **Identifying problems**
1. Verify is contests.json size match the quantity of .CSVs.
2.  Search for missing rows. By comparing the expected index (i + dataA_offset, ti get the starting contest_id value) and the actual index (row["contest_id"]) we can verify if any row is missing. We will deal with their filling a bit later. 
3. Verify if for each row contest_id, images and data have always the same number. 
4. Verify if the order of the datas are the same as indexes (ex: (id: 13) == (contest_id - dataA_offset))\
--> It is okay, since the 525.csv and 540.csv are also missing.

- **Define an absolute indexing system**\
Since we know the starting id dataA_startID, we can substract dataA_startID from all rows and get the "normalised" indexes. 
Also dataA is already normalised exluding missing rows, so we have to normalise without missing rows to be consistent with .csv.
From the previous tests (like id vs contest_id) we know that data is sorted by contest_id and hence by id itself.

    In 'src\utils\general_utils.py' are defined two methods to obtain the value of the index in absolute indexing system ('contest_index2absolute_index') or in contest indexing system ('absolute_index2contest_index').


#####  Construction of a New Funny Metric

To better identify which captions are truly funny, we designed a new *funny metric* that combines both the **proportion of votes** and the **popularity (number of votes)** of each caption.

#### 1. Weighted funny and unfunny ratios

We first compute the proportion of *funny* and *unfunny* votes relative to the total number of votes for each caption:

$$
\text{funny\_ratio} = \frac{N_{\text{funny}}}{N_{\text{total}}}
$$
$$
\text{unfunny\_ratio} = \frac{N_{\text{unfunny}}}{N_{\text{total}}}
$$

To give more importance to captions that received **more votes** (and are thus statistically more reliable), each ratio is weighted by the logarithm of the number of votes:

$$
\text{weighted\_funny} = \text{funny\_ratio} \times \log(1 + N_{\text{total}})
$$
$$
\text{weighted\_unfunny} = \text{unfunny\_ratio} \times \log(1 + N_{\text{total}})
$$

The logarithmic weighting ensures that captions with many votes are emphasized, while preventing those with extremely high vote counts from dominating the score.

#### 2. Standardization and combined score

We then normalize both weighted ratios using **z-scores** to make them comparable across captions:

$$
z_{\text{funny}} = \frac{\text{weighted\_funny} - \mu_{\text{funny}}}{\sigma_{\text{funny}}}
$$
$$
z_{\text{unfunny}} = \frac{\text{weighted\_unfunny} - \mu_{\text{unfunny}}}{\sigma_{\text{unfunny}}}
$$

Finally, the two standardized scores are combined into a **single composite score**:

$$
\text{combined\_score} = z_{\text{funny}} - z_{\text{unfunny}}
$$

A higher `combined_score` indicates captions that are **consistently rated funny** and **supported by a sufficient number of votes**.

#### 3. Ranking

All captions are then ranked according to this score:

$$
\text{rank\_funny} = \text{rank}(-\text{combined\_score})
$$

The highest `combined_score` (most funny) receives rank 1.


## 1.2. Gathering of other datasets

#### Temporal dataset augmentation :

The following webpage has **dates of some of the contests**. We add it to the **dataC** table as a new column:
"https://nextml.github.io/caption-contest-data/"

The next steps are followed, all in the same block of code so it can be re-run without issue.
1. Read the webpage, and get a "contest_id" for each image: Initially, the name of each contest is only given as "### Dashboard". Removing the "Dashboard" from the name. To fit with the previous format of dataC, reseting the index. We need to watch out, 540 exists in the new table. We need to remove it.
2. Additionally, the date here is the day the finalist was announced, not the date the cartoon came out... something to keep in mind.
3. Clean the format of the dates. Sometimes there is an "estimated" keyword, sometimes there is two dates, and sometimes, the year is missing. When there are two dates, I only keep the last date. When a year is missing from a date, look at the previous entry and take the year from there.
4. Convert the 'date' column of the dates_table dataframe to a correct date format by using pd.to_datetime.
5. The dates are prepared now and can be merged with our dataset.

#### Occupations dataset augmentation :

TO BE WRITTEN

#### Gender dataset augmentation :

TO BE WRITTEN

## 1.3. Executing the DataPreparation.ipynb file

This cell executes all what's been described above. We did not want to copy paste the code here to keep this result file clean, but for any verification about the code, please refer to ```src\data\DataPreparation.py```

In [5]:
%run {DATA_PREPARATION_PY_PATH.resolve()}

✅ Root folder detected at: C:\Users\cyrie\OneDrive\Documents\EPFL\2025_MA_1\ADA\ada-2025-project-adacore42\src\data


## 1.4. Descriptive statistic tasks

In [6]:
# Pickle datapreprocessed file loading
with open(STORED_DATAPREP_PKL_PATH, "rb") as f:
    data = pickle.load(f)

# Objects extraction
dataA = data['dataA']
dataC = data['dataC']
dataA_startID = data['dataA_startID']
dataA_endID = data['dataA_endID']
dataC_lastGoodID = data['dataC_lastGoodID']

In [9]:
#TODO

What I did as basic analysis (cyrielle), maybe we can take some plots of it ?

 #### - **On 1 single cartoon (dataA[108])** 

 *1.1: avg score of the cartoon vs. number of votes*\
 *1.2: histogram of avg score of all captions proposed + hypothesis test : normal distribution?*\
 *1.3: mean caption score vs. descending rank*\
 *1.4: number of votes vs. descending rank*\
 *1.5: histogram : 'not_funny'/'somewhat_funny'/'funny' ratios among all captions, for the cartoon*\
 *1.6: Text processing : TextBlob to obtain  a DataFrame of captions caracteristics ('tags', 'nouns', 'verbs', 'polarity', 'subjectivity')*\
 &emsp;&emsp; *1.6.1: histogram polarity and subjectivity of the cartoon*\
 &emsp;&emsp; *1.6.2: Top10 of the cartoon's most cited nouns*\
 &emsp;&emsp; *1.6.3: Top10 of the cartoon's most frequent verbs* (to be improved)\


 #### - **Plots/stats on all cartoons (dataC)** 

 *2.1: General statistics*\
 *2.2: Number of captions proposed vs. cartoon contest's id \
  &emsp; + Number of votes vs. cartoon contest's id*\
 *2.3: histogram : count of cartoons vs. number of votes \
  &emsp; + histogram : count of cartoons vs. number of captions proposed*\
 *2.4: Identify the most frequent visual themes among all cartoons*\
 &emsp;&emsp; *2.4.1: Top50 most cited Locations*\
 &emsp;&emsp; *2.4.2: Top50 most cited Locations (grouped by category)*\
 &emsp;&emsp; *2.4.3: Top10 most asked questions*\
 &emsp;&emsp; *2.4.4: Top10 most used question's W-words*\
 &emsp;&emsp; *2.4.5: Top40 most used verbs* (to be improved)\


## 1.5. Building usefull metrics

#### Similarity metric

About the semantic embedding model used : The all-MiniLM-L6-v2 is a lightweight sentence-transformers model: It maps sentences & paragraphs to a 384 dimensional dense vector space (embeddings). It is designed for natural language understanding tasks like semantic search, clustering, and similarity comparisons, offering a balance of high performance and computational efficiency.\
Requirements:
*pip install -U sentence-transformers*

In [28]:
from src.models.similarity_analysis import SimilarityModel, CaptionClustering, SimilarHumorAnalysis


df = dataA[77]


# 1. Similarity
sim_model = SimilarityModel(model_name='all-MiniLM-L6-v2')

# Similarity matrix
semantic_sim = sim_model.compute_semantic_similarity(df)
structure_sim = sim_model.compute_structure_similarity(df)
combined_sim = sim_model.compute_combined_similarity(semantic_sim, structure_sim, semantic_weight=0.7, structural_weight=0.3)

# Visualisation of similarity
sim_model.plot_similarity_matrix(df, semantic_weight=0.7, structural_weight=0.3)


# 2.Clustering
cluster_model = CaptionClustering(model_name='all-MiniLM-L6-v2')
cluster_labels, embeddings = cluster_model.cluster_captions(df, n_clusters=10)
df_clusters, embeddings = cluster_model.UMAP_reduction(df, n_clusters=10)


# 3. Analysis of humour scores within a similarity cluster
humor_analysis = SimilarHumorAnalysis(model_name='all-MiniLM-L6-v2')

corr = humor_analysis.scores_correlation(df, text_col='caption', humor_col='mean', semantic_weight=0.7, structural_weight=0.3)
print("Corrélations humour/similarité :", corr)

humor_analysis.plot_scores_correlation()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

TypeError: can't convert mps:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

##### From those diverse steps, statistic results and metric tools, we can now dive into our data story, yay !

<center>
    <img src="data/newyorker_caption_contest_virgin/images/612.jpg" alt="New Yorker Cartoon" style="width:350px; height:auto; border-radius:5px; box-shadow: 0 4px 8px rgba(0,0,0,0.1);">
</center>

# 2 Narrative Flow

## 2.1. Axis 1: Professions, Politics, and Power

**Professions in Humor:**  
Which jobs are depicted most often? Which are ridiculed vs. admired? What stereotypes are recurrent (e.g., lawyers as tricksters, doctors as saviors)?  

**Politics in Humor:**  
Do captions reflect partisan leanings (Democrat vs. Republican) or mock political figures more broadly? Are political jokes rated differently?  

**Interplay:**  
Professions like politicians or lawyers sit at the crossroads of both — this axis highlights how authority and social roles are viewed through humor.

### Plots / Stats

- **Bar / Word Clouds:** Frequency of professions mentioned in captions (“doctor,” “lawyer,” “politician”).  
- **Histograms / Line Plots:** Frequency of professions across time.  
- **Grouped Bar Charts:** Average “funniness” scores by profession category (healthcare, law, politics, education, etc.).  
- **Heatmaps:** Cross-tab professions × sentiment (positive / negative / neutral).  
- **Cartoon + Caption Samples:** A few annotated cartoons showing how professions are ridiculed (adds storytelling color).  

**For politics:**
- Timeline of mentions of political figures / parties.  
- Sentiment distribution around Democrats vs. Republicans.  
- Example “political joke clusters” side by side with major events (e.g., elections).  

**Statistical Tests / Methods:**
- *t-tests / z-tests* → Compare funniness scores of politicians vs. other professions.  
- *Multiple hypothesis testing (FDR/BH)* → Control for comparisons across 30+ job categories.  
- *Network graphs* → Co-occurrence of profession keywords with stereotypes (“lawyer–money,” “doctor–death”).  
- *Linear regression / lmplot* → Test if political humor ratings rise around elections.  
- *Pearsonr / Spearmanr* → Correlation between real-world political cycles and joke frequency.  

---

## Preliminary Research

---
---

## 2. Humor in Context: Time, News, and Global Events

**Temporal Shifts:**  
How do joke styles evolve over decades of contests?  

**Event-Driven Humor:**  
Are certain joke clusters (e.g., pandemic jokes, economic crisis humor) tied to real-world events?  

**Editorial & Cultural Climate:**  
To what extent might editorial influence what becomes “funny” at a given moment?

### Plots / Stats

- **Timeline Charts:** Average funniness scores across years (detecting peaks/troughs).  
- **Cluster Maps (t-SNE / UMAP):** Humor types (e.g., puns, absurdity, political jokes) clustered and color-coded by time period.  
  - *(PCA / scatter plot matrix → map high-dimensional humor features into 2D time-evolving plots.)*  
- **Overlay with Historical Events:** Vertical lines marking pandemics, recessions, elections, etc.  
- **Rolling Average Trends:** How often humor about specific topics (health, money, environment) surfaces year by year.  
- **Line Plots / Stacked Plots:** Share of humor categories over decades.  
- **Clustering (k-means, hierarchical):** Humor style clusters from text embeddings.  
- **Jointplot:** Caption length vs. funniness, with year as hue.  
- **kstest:** Test if distributions of funniness scores shift significantly during crises vs. stable periods.  
- **Causality tests (Granger):** Do global events precede spikes in political/taboo humor?  
- **Regression with time covariates:** Predict funniness as function of time + event markers.  

---

## Preliminary Research

---
---

## 3. Social Representations and Boundaries of Humor

**Gender and Representation:**  
Who appears in the cartoons? How are men vs. women depicted (professional vs. domestic roles)?  
Do captions reinforce stereotypes, and does the audience reward or punish them?  

**Taboo Themes:**  
How often do jokes about religion, sex, death, or violence appear? Are they rated as funnier or riskier depending on the era?  

**Audience Sensitivity:**  
Do taboos lose their edge over time, or does shock humor stay consistently popular?

### Plots / Stats

- **Gender Representation Bars:** Share of male vs. female figures in images.  
- **Role Distribution Sankey Diagram:** Flow from gender → depicted roles (domestic, professional, heroic, villainous).  
- **Caption Analysis Word Clouds:** Gendered terms (e.g., “wife,” “husband,” “boss,” “nurse”) and their co-occurrences.  
- **Taboo Radar Chart:** Relative frequency of taboo themes (death, religion, sex, violence, race) across decades.  
- **Boxplots:** Funniness scores of taboo vs. non-taboo captions.  
- **Annotated Timeline:** Spikes in taboo humor aligned with real-world crises or cultural shifts.  

**Statistical Tests:**
- *t-tests / binomial-tests* → Gender differences in roles; taboo vs. non-taboo funniness.  
- *Spearman correlation* → Taboo frequency vs. political/economic crises.  
- *CCDF* → Heavy-tailed popularity of taboo jokes (shock jokes succeed rarely but spectacularly).  

---


## Preliminary Research

---
---

## 4. Cognitive and Cultural Biases in Humor

**Attentional Biases:**  
Do certain visuals (talking animals, surreal objects) attract more humorous captions? Do raters become “bored” after reading too many captions?  

**Judgment Biases:**  
Are logical vs. emotional jokes received differently? Do certain rhetorical structures (puns, absurdities) align with higher scores?  

**Cultural Biases:**  
Do references to celebrities, brands, or memes outperform timeless jokes? How do culturally specific jokes age across time?

### Plots / Stats

- **Voting Distribution Curves:** Detect the “bored-rater effect” (scores declining as voters progress through captions).  
- **Scatterplots / regplots:** Visual oddity (unusual objects, surreal images) vs. funniness scores.  
- **Polarity / Subjectivity Histograms:** Comparing logical vs. emotional jokes.  
- **Cultural Reference Heatmap:** Proper nouns (brands, celebrities, memes) frequency and funniness.  
- **Side-by-Side Caption Examples:** One “timeless” vs. one “culture-specific” joke, annotated to show differences.  
- **Spearmanr / Pearsonr:** Test correlation between rating order and score.  
- **Histograms / PDF, CDF:** Distribution of culturally referential vs. non-referential captions.  
- **Log / semilog plots:** Detect heavy-tailed patterns (a few jokes get very high scores).  
- **Logistic regression:** Predict caption success from presence of cultural references.  
- **Unsupervised clustering:** Group captions by bias-driven humor style.  
- **Network graph:** Links between cultural references (brands / celebrities) and funniness.

---

## Preliminary Research

# What is the answer to ADA, the universe and everything ?

In [12]:
import random

answer = None
number_of_tries = 0
while answer != 42:
    # Generate one random number
    answer = random.randint(0, 1_000_000)
    number_of_tries += 1

print("Number of TEEEEEENTATIVES:", number_of_tries, end="\r")
print("\nThe answer to ADA, the universe and everything is:", answer)


Number of TEEEEEENTATIVES: 331677
The answer to ADA, the universe and everything is: 42
